## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


In [2]:
# Computing the camera calibration matrix and distortion coefficients given a set of chessboard images.

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt 

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nx = 9 # the number of inside corners in x
ny = 6 # the number of inside corners in y
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/*.jpg')
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        cv2.imshow('img',img)
        # cv2.waitKey(500)
cv2.destroyAllWindows()

In [34]:
# Applying distortion correction to raw images and perspective transform
images = glob.glob('./camera_cal/*.jpg')
offset = 100
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_size = (gray.shape[1], gray.shape[0])


    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    if ret == True:
    # Applying distortion correction
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1],None, None)
        undist = cv2.undistort(img,  mtx, dist, None, mtx)

        # Applying perspective transform
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], [img_size[0]-offset, img_size[1]-offset],[offset, img_size[1]-offset]])
        perscpective_transform_mat = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(undist, perscpective_transform_mat, img_size, flags=cv2.INTER_LINEAR)

        cv2.imshow('img',warped)
        cv2.waitKey(500)
        name = "./cal_out" + fname.replace('./camera_cal', '')
        cv2.imwrite( name, warped )
cv2.destroyAllWindows()

print(img.shape)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
print(gray.shape)



(720, 1280, 3)
(720, 1280)


In [ ]:
# Applying a perspective transform to rectify binary image.



In [42]:
# x and y sobel Gradients
def mag_gradient(img, sobel_kernel, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Sobel x gradients
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    # Sobel Y gradients
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    grad_mag = np.sqrt(sobel_x**2 + sobel_y**2)
    # Rescale to 8 bit
    scale_factor = np.max(grad_mag)/255 
    grad_mag = (grad_mag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(grad_mag)
    binary_output[(grad_mag >= mag_thresh[0]) & (grad_mag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# S-channel thresholding
def hls_threshold(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_s_output = np.zeros_like(s_channel)
    binary_s_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1

    return s_channel, binary_s_output

In [43]:
# Use color transforms, gradients, etc., to create a thresholded binary image.
thresh=(90, 255)
images = glob.glob('./test_images/*.jpg')
f, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2, 2, figsize=(24, 9))
mag_thresh = (30,100)
for fname in images:
    img = mpimg.imread(fname) #RGB image

    mag_binary = mag_gradient(img, 3, mag_thresh)
    s_channel, binary_s_output= hls_threshold(img)
    # hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # s_channel = hls[:,:,2]
    # binary_s_output = np.zeros_like(s_channel)
    # binary_s_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1

    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(s_channel, cmap='gray')
    ax2.set_title('S Channel', fontsize=50)
    ax3.imshow(mag_binary, cmap='gray')
    ax3.set_title('Gradient', fontsize=50)
    ax4.imshow(binary_s_output, cmap='gray')
    ax4.set_title('Thresholded S', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.pause(1)
    print(fname)
plt.close('all')



./test_images/test3.jpg
./test_images/test2.jpg
./test_images/test5.jpg
./test_images/test4.jpg
./test_images/test6.jpg
./test_images/test1.jpg
./test_images/straight_lines2.jpg
./test_images/straight_lines1.jpg
